In [6]:
%%writefile create_db.hql

DROP DATABASE IF EXISTS stackoverflow CASCADE;
CREATE DATABASE stackoverflow LOCATION '/user/jovyan/store/stackoverflow';
USE stackoverflow;

Overwriting create_db.hql


In [7]:
!hive -f create_db.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 0.592 seconds
OK
Time taken: 0.244 seconds
OK
Time taken: 0.007 seconds


In [58]:
%%writefile external_table.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

DROP TABLE IF EXISTS posts_sample_external;

CREATE EXTERNAL TABLE posts_sample_external (
    id INT,
    post_type_id TINYINT,
    date STRING,
    owner_user_id INT,
    parent_id INT,
    score INT,
    favorite_count INT,
    tags STRING    
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe' WITH SERDEPROPERTIES
('input.regex'='.*(?=\\bId=\"(\\d+)\")(?=.*\\bPostTypeId=\"(\\d+)\")(?=.*\\bCreationDate=\"(.{23})\")(?=.*\\bOwnerUserId=\"(\\d+)\")?(?=.*\\bParentId=\"(\\d+)\")?(?=.*\\bScore=\"(\\d+)\")(?=.*\\bFavoriteCount=\"(\\d+)\")?(?=.*\\bTags=\"(&lt\;.*gt\;)\")?.*')
LOCATION '/data/stackexchange100/posts';


Overwriting external_table.hql


In [59]:
!hive -f external_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
OK
Time taken: 0.438 seconds
OK
Time taken: 0.435 seconds


In [145]:
%%writefile sample.hql

DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (
    id INT,
    post_type_id TINYINT,
    owner_user_id INT,
    parent_id INT,
    score INT,
    favorite_count INT,
    tags ARRAY<STRING>
    
) 
PARTITIONED BY (fix INT, year STRING, month STRING)
STORED AS SEQUENCEFILE LOCATION '/user/jovyan/store/sample';

FROM posts_sample_external se
INSERT OVERWRITE TABLE posts_sample PARTITION(fix=1, year, month)
    select se.id, se.post_type_id, se.owner_user_id, se.parent_id, se.score, se.favorite_count,split(se.tags,"&lt\;|&gt\;"),substr(se.date,0,4), substr(se.date,6,2)
    WHERE se.id IS NOT NULL;


Overwriting sample.hql


In [142]:
!hive -f sample.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.095 seconds
OK
Time taken: 0.342 seconds
Query ID = jovyan_20171206125353_de88d4f5-74ee-4599-a2de-67e1c1629dc0
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1512550374499_0022, Tracking URL = http://57308a9d5017:8088/proxy/application_1512550374499_0022/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1512550374499_0022
Hadoop job information for Stage-1: number of mappers: 2; number of reducers: 0
2017-12-06 12:53:21,602 Stage-1 map = 0%,  reduce = 0%
2017-12-06 12:53:38,324 Stage-1 map = 14%,  reduce = 0%, Cumulative CPU 24.68 sec
2017-12-06 12:53:44,471 Stage-1 map = 25%,  reduce = 0%, Cumulative CPU 36.02 sec
2017-12-06 12:53:49,600 Stage-1 map = 30%,  reduce = 0%, Cumulative CPU 41.83 sec
2017-12-06 12:53:50,625 Stage-1 map = 35%,  reduce = 0%, Cu

Moving data to: hdfs://localhost:9000/user/jovyan/store/sample/fix=1/.hive-staging_hive_2017-12-06_12-53-15_421_5979680280931147311-1/-ext-10000/year=2011/month=07
Moving data to: hdfs://localhost:9000/user/jovyan/store/sample/fix=1/.hive-staging_hive_2017-12-06_12-53-15_421_5979680280931147311-1/-ext-10000/year=2011/month=08
Moving data to: hdfs://localhost:9000/user/jovyan/store/sample/fix=1/.hive-staging_hive_2017-12-06_12-53-15_421_5979680280931147311-1/-ext-10000/year=2011/month=09
Moving data to: hdfs://localhost:9000/user/jovyan/store/sample/fix=1/.hive-staging_hive_2017-12-06_12-53-15_421_5979680280931147311-1/-ext-10000/year=2011/month=10
Moving data to: hdfs://localhost:9000/user/jovyan/store/sample/fix=1/.hive-staging_hive_2017-12-06_12-53-15_421_5979680280931147311-1/-ext-10000/year=2011/month=11
Moving data to: hdfs://localhost:9000/user/jovyan/store/sample/fix=1/.hive-staging_hive_2017-12-06_12-53-15_421_5979680280931147311-1/-ext-10000/year=2011/month=12
Moving data to: 

Moving data to: hdfs://localhost:9000/user/jovyan/store/sample/fix=1/.hive-staging_hive_2017-12-06_12-53-15_421_5979680280931147311-1/-ext-10000/year=2016/month=01
Moving data to: hdfs://localhost:9000/user/jovyan/store/sample/fix=1/.hive-staging_hive_2017-12-06_12-53-15_421_5979680280931147311-1/-ext-10000/year=2016/month=02
Moving data to: hdfs://localhost:9000/user/jovyan/store/sample/fix=1/.hive-staging_hive_2017-12-06_12-53-15_421_5979680280931147311-1/-ext-10000/year=2016/month=03
Moving data to: hdfs://localhost:9000/user/jovyan/store/sample/fix=1/.hive-staging_hive_2017-12-06_12-53-15_421_5979680280931147311-1/-ext-10000/year=2016/month=04
Moving data to: hdfs://localhost:9000/user/jovyan/store/sample/fix=1/.hive-staging_hive_2017-12-06_12-53-15_421_5979680280931147311-1/-ext-10000/year=2016/month=05
Moving data to: hdfs://localhost:9000/user/jovyan/store/sample/fix=1/.hive-staging_hive_2017-12-06_12-53-15_421_5979680280931147311-1/-ext-10000/year=2016/month=06
Moving data to: 

Partition default.posts_sample{fix=1, year=2010, month=09} stats: [numFiles=1, numRows=1791, totalSize=93314, rawDataSize=69044]
Partition default.posts_sample{fix=1, year=2010, month=10} stats: [numFiles=1, numRows=1875, totalSize=96652, rawDataSize=71250]
Partition default.posts_sample{fix=1, year=2010, month=11} stats: [numFiles=1, numRows=2054, totalSize=106800, rawDataSize=78971]
Partition default.posts_sample{fix=1, year=2010, month=12} stats: [numFiles=1, numRows=2049, totalSize=106167, rawDataSize=78423]
Partition default.posts_sample{fix=1, year=2011, month=01} stats: [numFiles=1, numRows=2328, totalSize=121707, rawDataSize=90176]
Partition default.posts_sample{fix=1, year=2011, month=02} stats: [numFiles=1, numRows=2387, totalSize=125444, rawDataSize=93106]
Partition default.posts_sample{fix=1, year=2011, month=03} stats: [numFiles=1, numRows=2942, totalSize=154494, rawDataSize=114661]
Partition default.posts_sample{fix=1, year=2011, month=04} stats: [numFiles=1, numRows=2765

MapReduce Jobs Launched: 
Stage-Stage-1: Map: 2   Cumulative CPU: 117.66 sec   HDFS Read: 493793931 HDFS Write: 17208671 SUCCESS
Stage-Stage-5: Map: 1   Cumulative CPU: 1.73 sec   HDFS Read: 263607 HDFS Write: 260312 SUCCESS
Total MapReduce CPU Time Spent: 1 minutes 59 seconds 390 msec
OK
Time taken: 136.922 seconds


In [147]:
%%writefile count.hql

INSERT OVERWRITE LOCAL DIRECTORY 'result' row format delimited fields terminated by '\t' stored as textfile
SELECT year,concat(year, "-", month),count(*) FROM posts_sample group by year,month LIMIT 3;

Overwriting count.hql


In [148]:

!hive -f count.hql
!cat result/*|head -1


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Query ID = jovyan_20171206125858_8f770834-49ad-4865-be7f-e52ef11159ed
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1512550374499_0026, Tracking URL = http://57308a9d5017:8088/proxy/application_1512550374499_0026/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1512550374499_0026
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2017-12-06 12:58:15,237 Stage-1 map = 0%,  reduce = 0%
2017-12-06 12:58:21,457 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 